In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv
# from config import api_key

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2023
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
pprint(response)

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'result': {'_links': {'next': '/api/3/action/datastore_search?limit=1000&q=2023&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&offset=1000',
                       'start': '/api/3/action/datastore_search?limit=1000&q=2023&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991'},
            'fields': [{'id': '_id', 'type': 'int'},
                       {'id': 'OBJECTID', 'type': 'text'},
                       {'id': 'Year', 'type': 'text'},
                       {'id': 'State', 'type': 'text'},
                       {'id': 'Agency', 'type': 'text'},
                       {'id': 'Unit ID', 'type': 'text'},
                       {'id': 'Fire Name', 'type': 'text'},
                       {'id': 'Local Incident Number', 'type': 'text'},
                       {'id': 'Alarm Date', 'type': 'text'},
                       {'id': 'Containment Date', 'type': 'text'},
                       {'id': 'Cause', 'type': 

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2023 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2023.columns if col not in columns_to_keep]
df_2023 = df_2023.drop(columns=columns_to_drop)
df_2023 = df_2023.dropna(subset=['Containment Date'])
df_2023.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2023,8/17/2023 12:00:00 AM,10/25/2023 12:00:00 AM,1,12152.45
1,2023,8/17/2023 12:00:00 AM,9/28/2023 12:00:00 AM,1,95.91678
2,2023,8/19/2023 12:00:00 AM,9/2/2023 12:00:00 AM,1,20.01698
3,2023,8/15/2023 12:00:00 AM,9/15/2023 12:00:00 AM,1,1684.73
4,2023,8/15/2023 12:00:00 AM,10/25/2023 12:00:00 AM,1,34188.45


In [4]:
alarm_date_23 = []
containment_date_23 = []
acres_23 = []
for index, row in df_2023.iterrows():
    alarm_date_23.append(row['Alarm Date'])
    containment_date_23.append(row['Containment Date'])
    acres_23.append(row['GIS Calculated Acres'])
alarm_date_23 = [dt.replace(' 12:00:00 AM', '') for dt in alarm_date_23]
containment_date_23 = [str(item) for item in containment_date_23]
containment_date_23 = [dt.replace(' 12:00:00 AM', '') for dt in containment_date_23]

In [5]:
df_2023['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2023['Alarm Date']]
df_2023['Containment Date'] = [str(item) for item in df_2023['Containment Date']]
df_2023['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2023['Containment Date']]
df_2023.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2023,8/17/2023,10/25/2023,1,12152.45
1,2023,8/17/2023,9/28/2023,1,95.91678
2,2023,8/19/2023,9/2/2023,1,20.01698
3,2023,8/15/2023,9/15/2023,1,1684.73
4,2023,8/15/2023,10/25/2023,1,34188.45


In [6]:
df_2023 = df_2023.rename(columns={'Cause': 'Cause #'})
df_2023['Cause'] = ''
df_2023['Cause #'] = df_2023['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2023['Cause'] = df_2023['Cause #'].apply(map_cause_to_meaning)
df_2023.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2023,8/17/2023,10/25/2023,1,12152.45,Lightning
1,2023,8/17/2023,9/28/2023,1,95.91678,Lightning
2,2023,8/19/2023,9/2/2023,1,20.01698,Lightning
3,2023,8/15/2023,9/15/2023,1,1684.73,Lightning
4,2023,8/15/2023,10/25/2023,1,34188.45,Lightning


In [7]:
df_2023.to_csv("outputs/fires_2023.csv",
                  encoding="utf-8", index=False, header=True)

In [8]:
df_2023['Alarm Date'] = pd.to_datetime(df_2023['Alarm Date'])
df_byalarm = df_2023.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,251,2023,2023-03-11,3/12/2023,14,15.25318,Unknown / Unidentified
1,161,2023,2023-04-06,4/6/2023,11,54.54718,Powerline
2,235,2023,2023-04-17,4/22/2023,10,62.93432,Vehicle
3,252,2023,2023-04-20,4/22/2023,14,19.89741,Unknown / Unidentified
4,188,2023,2023-04-26,5/11/2023,14,226.9751,Unknown / Unidentified


In [9]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2023-03-11 00:00:00')

In [10]:
df_2023['Containment Date'] = pd.to_datetime(df_2023['Containment Date'])
df_bycontainment = df_2023.sort_values(by='Containment Date', ascending = False).reset_index()
latest_containment = df_bycontainment['Containment Date'][0]
df_bycontainment.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,228,2023,2023-08-15,2024-01-09,14,76.7887,Unknown / Unidentified
1,199,2023,2023-08-15,2023-12-31,14,31.21948,Unknown / Unidentified
2,213,2023,2023-12-22,2023-12-22,14,38.60516,Unknown / Unidentified
3,141,2023,2023-12-16,2023-12-16,10,3.066564,Vehicle
4,100,2023,2023-12-15,2023-12-16,14,169.973,Unknown / Unidentified


In [11]:
latest_containment

Timestamp('2024-01-09 00:00:00')

In [26]:
output_path = os.path.join("outputs", "timeline_dates_2023.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])